In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.simplefilter('ignore', UserWarning)
warnings.simplefilter('ignore', DeprecationWarning)
import sys
sys.path.append('/home/ubuntu/fastai/')
from fastai.imports import *

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [16]:
PATH = "/home/ubuntu/data/imdb/"

TRN_PATH = "train/all/"
VAL_PATH = "test/all/"
TRN = f"{PATH}{TRN_PATH}"
VAL = f"{PATH}{VAL_PATH}"

%ls {PATH}

README  imdb.vocab  imdbEr.txt  test/  train/


In [17]:
trn_files = !ls {TRN}
trn_files[:10]

['0_0.txt',
 '0_3.txt',
 '0_9.txt',
 '10000_0.txt',
 '10000_4.txt',
 '10000_8.txt',
 '10001_0.txt',
 '10001_10.txt',
 '10001_4.txt',
 '10002_0.txt']

In [22]:
review = !cat {TRN}{trn_files[6]}

In [24]:
!find {TRN} -name '*.txt' | xargs cat | wc -w

17486270


In [25]:
!find {VAL} -name '*.txt' | xargs cat | wc -w

5686609


In [27]:
spacy_tok = spacy.load('en')

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [28]:
''.join([sent.string.strip() for sent in spacy_tok(review[0])])

"Everybodyhasseen'BackToTheFuture,'right?WhetheryouLIKEthatmovieornot,you'veseenanexampleofhowtomakeatime-travelmoviework.Atorn-upposterfor'BackToTheFuture'showsupinthismovie,representing,perhapsunintentionally,whatthemakersof'Tangents'(aka'TimeChasers')didtothetime-travelformula.Thenagain,themovieclaimstohavebeenmadein1994,butitlooks--andsounds--likeitwasproducedatleasttenyearsearlier,somaybetheyachievedtime-travelafterall.<br/><br/>Startwithanintenselyunappealingleadingman.Imean,whatwomandoesn'tlovegangly,whiny,lantern-jawed,butt-chinned,mullet-menwithCoke-bottleglasses?Oh,noneofyou?Preparetotoughitout,ladies,cuzthat'swhatthismoviegivesyou.<br/><br/>Second,addaleadingladywho--whilenotentirelyunattractive--representsmany'80sclich��s:bighair,toomuchmakeup,twodifferentplaids,shoulderpads,acid-washedmom-jeans,etc.<br/><br/>ThrowinaMichaelMedvedlook-alikewhowearspinkblazerswithwhitepants,astunninglytransparentvillainwhotalkslikeMortimerSnerdandhasanofficethatlookslikeacircus-themedlibrary

In [29]:
TEXT = data.Field(lower=True, tokenize='spacy')

In [31]:
bs=64;bptt=70

In [32]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [38]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [39]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(4583, 37392, 1, 20540756)

In [40]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'in', 'it']

In [41]:
TEXT.vocab.stoi['the']

2

In [46]:
md.trn_ds[0].text[:12]

['picking',
 'up',
 'the',
 'jacket',
 'of',
 'this',
 'dvd',
 'in',
 'the',
 'video',
 'store',
 'i']

In [47]:
TEXT.numericalize([md.trn_ds[0].text[:12]])

tensor([[3701],
        [  68],
        [   2],
        [5641],
        [   7],
        [  13],
        [ 300],
        [  10],
        [   2],
        [ 392],
        [1089],
        [  12]], device='cuda:0')

In [48]:
next(iter(md.trn_dl))

(tensor([[ 3701,   297,     9,  ...,  1667,    12,    14],
         [   68, 13351,   359,  ...,    43,   173,    33],
         [    2,    51,    24,  ...,   472,     6,   324],
         ...,
         [   16,   137,   144,  ...,  1659,  1310,    13],
         [  192,     6,   112,  ...,   976,    17,    37],
         [  213,  3557,     8,  ...,     3,   201,  1306]], device='cuda:0'),
 tensor([   68, 13351,   359,  ...,     2,    17,    14], device='cuda:0'))

In [49]:
em_sz = 200
nh = 500
nl = 3

In [50]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [51]:
learner = md.get_model(opt_fn, em_sz, nh, nl, dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [ ]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

epoch      trn_loss   val_loss                                
    0      4.837104   4.722275  
 83%|████████▎ | 3809/4583 [17:37<03:34,  3.60it/s, loss=4.7] 

In [ ]:
learner.save_encoder('adam1_enc')

In [ ]:
learner.load_encoder('adam1_enc')

In [ ]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=10)

In [ ]:
learner.save_encoder('adam3_10_enc')

In [ ]:
learner.load_encoder('adam3_10_enc')

In [ ]:
math.exp(4.165)

In [ ]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [ ]:
m=learner.model
ss=""". So, it wasn't quite was I was expecting, but I really liked it anyway! The best"""
s = [spacy_tok(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

In [ ]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

In [ ]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

In [ ]:
print(ss,"\n")
for i in range(50):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

In [ ]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

In [ ]:
IMDB_LABEL = data.Field(sequential=False)
splits = torchtext.datasets.IMDB.splits(TEXT, IMDB_LABEL, 'data/')

In [ ]:
t = splits[0].examples[0]

In [ ]:
t.label, ' '.join(t.text[:16])

In [ ]:
md2 = TextData.from_splits(PATH, splits, bs)

In [ ]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder(f'adam3_10_enc')

In [ ]:
m3.clip=25.
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [ ]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

In [ ]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='imdb2')

In [ ]:
m3.load_cycle('imdb2', 4)

In [ ]:
accuracy_np(*m3.predict_with_targs())